## [5day - 1] Model Selection 모듈

*   모델 구성 및학습에 필요한 다양한 API 제공
*   데이터 세트 분리
*   교차 검증 분할 및 평가
*   하이퍼 파라미터튜닝

# 데이터 셋 분할의 필요성
* 학습 데이터셋으로 평가 수행시 부정확한(과도하게높은) 평가 결과 발생


In [1]:
# 라이브러리 불러오기
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

dt_clf.fit(train_data, train_label)
pred = dt_clf.predict(train_data)

print('prediction accuracy : ', accuracy_score(train_label, pred))

prediction accuracy :  1.0


# train_test_split() API - 데이터 셋의 분할(학습/ 테스트)을 쉽게 할 수 있는 API

* Argument(인자)

> • test_size: 테스트 데이터셋의 비율

> • train_size: 학습 데이터셋의 비율

> • shuffle: 데이터셋 분리 전 미리 섞을지 여부

> • random_state: 실행시 마다 동일한 난수를 생성하기 위한 값

* Return

> • 튜플 형태로 분할(학습/ 테스트)된 데이터 리턴

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, 
                                                    test_size=0.3, 
                                                    random_state=121)

* 분할된 데이터 중 학습데이터를 이용해 학습
* 분할된 데이터 중 테스트 데이터를 이용해 성능 검증


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [6]:
# Decision Tree 모델 객체 생성
dt_clf = DecisionTreeClassifier()

In [7]:
# Decision Tree 모델 객체 학습
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [8]:
# 테스트 데이터를 이용한 예측
pred = dt_clf.predict(X_test)

print('prediction accuracy : {}'.format(accuracy_score(y_test, pred)))

prediction accuracy : 0.9555555555555556


## KFold API - K겹 교차 검증을 쉽게 적용 할 수 있는 Cross Validation API

* Argument(인자)

> • n_splits: 생성하고자 하는 데이터 Fold의 개수

> • shuffle: 데이터 Fold 분리 전 미리 섞을지 여부

> • random_state: 실행시마다 동일한 난수를 생성하기 위한 값

In [9]:
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris

In [10]:
# Iris 데이터셋 로드 및 KFold 객체 생
iris = load_iris()
features = iris.data
label = iris.target
kfold = KFold(n_splits=5)

print('Iris Data Set Size : ', features.shape[0])

Iris Data Set Size :  150


In [11]:
# Decision Tree 객체 생성 및 Fold 별 지표 값 저장을 위한 리스트 생성
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

dt_clf = DecisionTreeClassifier(random_state=156)
cv_accuracy = []

In [12]:
fold_index = 0

for train_index, test_index in kfold.split(features):

    # KFold 객체의 split() 함수를 이용해 분할된 Fold 별 인덱스 생성
    # 생성된 인덱스를 이용해 데이터셋 분할
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    print(train_index)
    print(test_index)

    # 분할된 Fold를 이용하여 모델 학습 
    dt_clf.fit(X_train, y_train)

    # 성능 측정
    pred = dt_clf.predict(X_test)

    fold_index += 1

    # Fold 별정확도지표측정및cv_accuracy 리스트에추가
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print('\n#{0} fold accuracy : {1}, train size : {2}, val size : \
    {3}'.format(fold_index, accuracy, train_size, test_size))
    print('#{0} val index : {1}'.format(fold_index, test_index))

    cv_accuracy.append(accuracy)

    # cv_accuracy 리스트에 저장된 Fold 별 정확도 값을 이용해 평균 정확도 계산
    # 계산된 KFold 평가지표(평균정확도) 출력
    print('\n## avg val accuracy : ', np.mean(cv_accuracy))
    print('###################################################################')

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#1 fold accuracy : 1.0, train size : 120, val size :     30
#1 val index : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

## avg val accuracy :  1.0
###################################################################
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67